In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm
import seaborn as sns
import statsmodels.formula.api as smf
from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error
from sklearn import neighbors
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import ensemble
from scipy.stats import boxcox

import warnings
warnings.filterwarnings(action='ignore')


postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'fifa19'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

fifa = pd.read_sql_query('select * from fifa19',con=engine)

engine.dispose()

fifa.head(10)

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
5,183277,E. Hazard,27,https://cdn.sofifa.org/players/4/19/183277.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,91,Chelsea,https://cdn.sofifa.org/teams/2/light/5.png,...,91.0,34.0,27.0,22.0,11.0,12.0,6.0,8.0,8.0,€172.1M
6,177003,L. Modrić,32,https://cdn.sofifa.org/players/4/19/177003.png,Croatia,https://cdn.sofifa.org/flags/10.png,91,91,Real Madrid,https://cdn.sofifa.org/teams/2/light/243.png,...,84.0,60.0,76.0,73.0,13.0,9.0,7.0,14.0,9.0,€137.4M
7,176580,L. Suárez,31,https://cdn.sofifa.org/players/4/19/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,91,91,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,...,85.0,62.0,45.0,38.0,27.0,25.0,31.0,33.0,37.0,€164M
8,155862,Sergio Ramos,32,https://cdn.sofifa.org/players/4/19/155862.png,Spain,https://cdn.sofifa.org/flags/45.png,91,91,Real Madrid,https://cdn.sofifa.org/teams/2/light/243.png,...,82.0,87.0,92.0,91.0,11.0,8.0,9.0,7.0,11.0,€104.6M
9,200389,J. Oblak,25,https://cdn.sofifa.org/players/4/19/200389.png,Slovenia,https://cdn.sofifa.org/flags/44.png,90,93,Atlético Madrid,https://cdn.sofifa.org/teams/2/light/240.png,...,70.0,27.0,12.0,18.0,86.0,92.0,78.0,88.0,89.0,€144.5M


In [2]:
#Drop rows with >=10 NaNs
fifa= fifa.dropna(thresh=10)
fifa.describe()

,ID,Age,Overall,Potential,Special,International Reputation,Weak Foot,Skill Moves,Jersey Number,Crossing,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
count,18207.000000,18207.000000,18207.000000,18207.000000,18207.000000,18159.000000,18159.000000,18159.000000,18147.000000,18159.000000,...,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000
mean,214298.338606,25.122206,66.238699,71.307299,1597.809908,1.113222,2.947299,2.361308,19.546096,49.734181,...,48.548598,58.648274,47.281623,47.697836,45.661435,16.616223,16.391596,16.232061,16.388898,16.710887
std,29965.244204,4.669943,6.908930,6.136496,272.586016,0.394031,0.660456,0.756164,15.947765,18.364524,...,15.704053,11.436133,19.904397,21.664004,21.289135,17.695349,16.906900,16.502864,17.034669,17.955119
min,16.000000,16.000000,46.000000,48.000000,731.000000,1.000000,1.000000,1.000000,1.000000,5.000000,...,5.000000,3.000000,3.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,200315.500000,21.000000,62.000000,67.000000,1457.000000,1.000000,3.000000,2.000000,8.000000,38.000000,...,39.000000,51.000000,30.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,221759.000000,25.000000,66.000000,71.000000,1635.000000,1.000000,3.000000,2.000000,17.000000,54.000000,...,49.000000,60.000000,53.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,236529.500000,28.000000,71.000000,75.000000,1787.000000,1.000000,3.000000,3.000000,26.000000,64.000000,...,60.000000,67.000000,64.000000,66.000000,64.000000,14.000000,14.000000,14.000000,14.000000,14.000000
max,246620.000000,45.000000,94.000000,95.000000,2346.000000,5.000000,5.000000,5.000000,99.000000,93.000000,...,92.000000,96.000000,94.000000,93.000000,91.000000,90.000000,92.000000,91.000000,90.000000,94.000000


In [3]:
#Drop rows missing key statistics
null_rows=fifa.loc[fifa.Crossing.isnull()]
fifa=fifa.drop(null_rows.index)

In [4]:
#Drop features that do not correlate w/ target
fifa= fifa.drop(['Contract Valid Until', 'Height', 'ID', 'Club Logo', 'Release Clause', 'Club', 'Nationality', 'Flag', 'Joined', 'Name', 'Photo', 'Real Face', 'Jersey Number', 'Position', 'Value'], axis=1)

In [5]:
fifa=fifa.drop(['Wage'], axis=1)

In [6]:
positions=fifa.loc[:, 'LS':'RB']
positions

,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,...,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB
0,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,...,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2
1,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,...,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
2,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,...,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,...,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3
5,83+3,83+3,83+3,89+3,88+3,88+3,88+3,89+3,89+3,89+3,...,66+3,63+3,63+3,63+3,66+3,60+3,49+3,49+3,49+3,60+3
6,77+3,77+3,77+3,85+3,84+3,84+3,84+3,85+3,87+3,87+3,...,82+3,81+3,81+3,81+3,82+3,79+3,71+3,71+3,71+3,79+3
7,87+5,87+5,87+5,86+5,87+5,87+5,87+5,86+5,85+5,85+5,...,69+5,68+5,68+5,68+5,69+5,66+5,63+5,63+5,63+5,66+5
8,73+3,73+3,73+3,70+3,71+3,71+3,71+3,70+3,71+3,71+3,...,81+3,84+3,84+3,84+3,81+3,84+3,87+3,87+3,87+3,84+3
9,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [21]:
#Create new df using the product of each cell
summed_df = positions.applymap(lambda x: eval(x) if x else x)
summed_df= summed_df.fillna(summed_df.mean())
summed_df.shape

(18159, 26)

In [8]:
#Convert to numeric
fifa.Weight=fifa.Weight.str.strip('lbs')
fifa.Weight= pd.to_numeric(fifa.Weight)

In [9]:
#Defining featureset
X=fifa.drop('Overall', axis=1)

In [10]:
#Replace positional columns with cleaned version
X=X.drop(positions, axis=1)
X=pd.concat([X, summed_df], axis=1)
X=X.fillna(X.mean())

In [12]:
#Try Modelling with random forest regressor
X=pd.get_dummies(X)
Y=fifa['Overall']
rfr=ensemble.RandomForestRegressor(max_depth= 3, n_estimators=100)
rfr.fit(X, Y)

cross_val_score(rfr, X, Y, cv=10)

array([ -4.99809288, -10.07457044, -24.54514876, -32.22672983,
       -24.21732593, -37.28305822, -28.83550336, -20.27748614,
       -16.51819455, -11.34935462])

In [23]:
X.head()

,Age,Potential,Special,International Reputation,Weak Foot,Skill Moves,Weight,Crossing,Finishing,HeadingAccuracy,...,Loaned From_Vålerenga Fotball,Loaned From_Vélez Sarsfield,Loaned From_Walsall,Loaned From_Watford,Loaned From_West Bromwich Albion,Loaned From_West Ham United,Loaned From_Wigan Athletic,Loaned From_Wolverhampton Wanderers,Loaned From_Yokohama F. Marinos,Loaned From_Östersunds FK
0,31,94,2202,5.0,4.0,4.0,159,84.0,95.0,70.0,...,0,0,0,0,0,0,0,0,0,0
1,33,94,2228,5.0,4.0,5.0,183,84.0,94.0,89.0,...,0,0,0,0,0,0,0,0,0,0
2,26,93,2143,5.0,5.0,5.0,150,79.0,87.0,62.0,...,0,0,0,0,0,0,0,0,0,0
3,27,93,1471,4.0,3.0,1.0,168,17.0,13.0,21.0,...,0,0,0,0,0,0,0,0,0,0
4,27,92,2281,4.0,5.0,4.0,154,93.0,82.0,55.0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
#Split and analyze results
X_train, X_test, y_train, y_test= train_test_split(X, Y, test_size= 0.20, random_state=498)

rfr=ensemble.RandomForestRegressor(max_depth= 6, n_estimators=100)

rfr.fit(X_train, y_train)

# We are making predictions here
y_preds_test = rfr.predict(X_test)

print("R-squared of the model in training set is: {}".format(rfr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(rfr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

R-squared of the model in training set is: 0.9379646805797227
-----Test set statistics-----
R-squared of the model in test set is: 0.9314715883552714
Mean absolute error of the prediction is: 1.2817729466110128
Mean squared error of the prediction is: 3.3039173314212302
Root mean squared error of the prediction is: 1.8176681026582466
Mean absolute percentage error of the prediction is: 2.0223177237303758
--- 0.00012993812561035156 seconds ---


In [36]:
#Use single decision tree for comparison
from sklearn.tree import DecisionTreeRegressor

dtr=DecisionTreeRegressor(max_depth=10)

dtr.fit(X_train, y_train)

# We are making predictions here
y_preds_test = dtr.predict(X_test)

print("R-squared of the model in training set is: {}".format(dtr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(dtr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

R-squared of the model in training set is: 0.9742651943183298
-----Test set statistics-----
R-squared of the model in test set is: 0.9561990455344814
Mean absolute error of the prediction is: 0.9579496588876006
Mean squared error of the prediction is: 2.111747946846669
Root mean squared error of the prediction is: 1.4531854481953324
Mean absolute percentage error of the prediction is: 1.5093473985524863
--- 7.295608520507812e-05 seconds ---


Since random forest models are better at reducing _variance_ rather than bias, it is not surprising to see the decision tree outperformed the random forest on test data accuracy. It is much faster, and more easy to interpret as you may render a decision tree and see the exact criteria for the decisions made. However, due to the structure of a single decision tree, it may be prone to overfitting the noise in the dataset-since there are such low totals of samples at each leaf node 10 layers down, predictions may be completed with very few samples satisfying each and every condition met. 

Random forests will help correct this problem, as they select samples of observations of the data and random subsets of features for each tree to combat errors of variance. By reducing depth and increasing the number of trees, we may reduce both errors of variance _and_ bias to create a more predictive model that performs consistently across training and test sets. 